In [1]:
#local files(same folder)
from docker.vars.hbasevar import *
#Access outside docker
hbHost= '127.0.0.1'

#python libraries
import ipywidgets as widgets
from IPython.display import clear_output
import happybase as hb



In [2]:
#Front-end design
a=widgets.Text(
    placeholder='Please type word to search',
    description='Word:',
    disabled=False
)

button=widgets.Button(
    description='Submit Word',
    disabled=False,
    button_style='info',
    tooltip='When clicking this button you can see the results of the search',
    icon='check'
)

In [3]:
subjectQ = True

#Function to query hbase
def consultTable(event):
    clear_output()
    display(a)
    display(button)
    #Check empty word
    if a.value == "":
        print("Please type word to search")
    else:
        #TODO: Check HBase connection
        #TODO: Check if HBase table/columns exist
        #Create connection, send query
        conn = hb.Connection(hbHost,hbPort)
        
        if subjectQ:
            #Subject query
            hbaseFilter = "SingleColumnValueFilter ('metadata','subject',=,'substring:{}')".format(a.value)
            results=conn.table(hbTableName).scan(columns=[b'metadata:subject',b'metadata:label']
                                                 , filter=hbaseFilter)
            #Print results
            for key, data in results:
                print('File: ' + key.decode('utf-8'))
                print('')
                print('    Subject: ' + data[b'metadata:subject'].decode('utf-8') )
                print('    Label: ' + data[b'metadata:label'].decode('utf-8'))
                print('')
                print('')
        
        else:
            
            #Content and metadata query
            hbaseCFilter = "(SingleColumnValueFilter ('content','raw',=,'substring:{}'))".format(a.value)
            hbaseMFilter = "(SingleColumnValueFilter ('metadata','raw',=,'substring:{}'))".format(a.value)
            hbaseCMFilter = hbaseCFilter + " OR " + hbaseMFilter
            results=conn.table(hbTableName).scan(columns=[b'content:raw',b'metadata:raw',b'metadata:label']
                                                 , filter=hbaseCMFilter)
            #Print results
            for key, data in results:
                print('File: ' + key.decode('utf-8'))
                print('')
                print('    Metadata: ' + data[b'metadata:raw'].decode('utf-8') )
                print('')
                print('    Content: ' + data[b'content:raw'].decode('utf-8') )
                print('')
                print('    Label: ' + data[b'metadata:label'].decode('utf-8'))
                print('')
                print('')
            
        #Close connection
        conn.close()

#link button to hbase function
button.on_click(consultTable)

In [4]:
#Display
display(a)
display(button)

Text(value='Trading', description='Word:', placeholder='Please type word to search')

Button(button_style='info', description='Submit Word', icon='check', style=ButtonStyle(), tooltip='When clicki…

File: Data/SPAM/BG/2004/09/1096397598.6542_208.txt

    Subject: Re:ftfwj,PennySt0ck Trading Report
    Label: SPAM


